In [1]:
import mediapipe as mp
import cv2
import numpy as np
import time
import face_knn



In [12]:
face_assessment = face_knn.Face_Assessment("../../profiling_calib/calib_records/calibration_Yassine Chahboun.json")


In [13]:
def calculate_distance(point1, point2):
    return np.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2+(point1.z - point2.z)**2)

def get_eyes_lips_relative_distance(face_landmarks):
    if not face_landmarks:
        return {
            "lips": -1,
            "left_eye": -1,
            "right_eye": -1
        }

    #relevant points for lips, left eye, and right eye
    upper_lip = face_landmarks.landmark[13]
    bottom_lip = face_landmarks.landmark[14]
    upper_left_eye_point = face_landmarks.landmark[386]
    bottom_left_eye_point = face_landmarks.landmark[374]
    upper_right_eye_points = face_landmarks.landmark[159]
    bottom_right_eye_points = face_landmarks.landmark[145]
    upper_face = face_landmarks.landmark[10]
    bottom_face = face_landmarks.landmark[152]

    #Relative distances
    lips_distance = calculate_distance(upper_lip, bottom_lip) / calculate_distance(upper_face, bottom_face)
    left_eye_distance = calculate_distance(upper_left_eye_point, bottom_left_eye_point) / calculate_distance(upper_face, bottom_face)
    right_eye_distance = calculate_distance(upper_right_eye_points, bottom_right_eye_points) / calculate_distance(upper_face, bottom_face)

    return {
        "lips": lips_distance,
        "left_eye": left_eye_distance,
        "right_eye": right_eye_distance
    }


In [14]:
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils

In [16]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        res = holistic.process(frame)
        frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            frame,
            res.face_landmarks,
            mp_holistic.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
        if res.face_landmarks:
            dists = get_eyes_lips_relative_distance(res.face_landmarks)
            print(dists["lips"],dists["left_eye"],dists["right_eye"])
            print(face_assessment.get_results(
                lips=dists["lips"],
                left_eye=dists["left_eye"],
                right_eye=dists["right_eye"]
            ))

        cv2.imshow("frame",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()
    

0.0045025378940320125 0.04449795165320611 0.0450495949871882
['TALKING']
0.005594727790983354 0.044397648354596725 0.044504195740800556
['TALKING']
0.006871897676705887 0.04742954622590362 0.048692969287414185
['TALKING']
0.005659753858665771 0.046401921155698904 0.046348984617804775
['TALKING']
0.007936732428484323 0.043046056676369675 0.043775196074760145
['TALKING']
0.0077720232499967025 0.04468739509094412 0.04466823373333346
['TALKING']
0.005142570022886655 0.045145784508721365 0.044656583689356814
['TALKING']
0.009500831071784862 0.04201271485989882 0.04203999044506642
['TALKING']
0.005613103188456511 0.04947177355365951 0.04833331216901029
['TALKING']
0.0073855257293772535 0.04633477972942686 0.04521069576151384
['TALKING']
0.007509339440085274 0.044721603871622016 0.043420012697004344
['TALKING']
0.006569841696348603 0.04728648614256432 0.04511148781132014
['TALKING']
0.006367473035244749 0.0465279945598885 0.04432683063846152
['TALKING']
0.005728521457822157 0.0475863361645594

: 

In [8]:
cap.release()
cv2.destroyAllWindows()